In [10]:
library(metacell)
library(GEOquery)
library(googlesheets4)
library(googledrive)
library(devtools)
library(dplyr)
library(tibble)
library(ggplot2)
library(tidyverse)
library(tgstat)
library(tgconfig)
library(liana)
library(SingleCellExperiment)
library(Seurat)
library(magrittr)

In [13]:
?liana::liana_aggregate

In [3]:
show_resources()

[1] "Default"          "Consensus"        "Baccin2019"       "CellCall"        
 [5] "CellChatDB"       "Cellinker"        "CellPhoneDB"      "CellTalkDB"      
 [9] "connectomeDB2020" "EMBRACE"          "Guide2Pharma"     "HPMR"            
[13] "ICELLNET"         "iTALK"            "Kirouac2010"      "LRdb"            
[17] "Ramilowski2015"   "OmniPath"

In [4]:
show_methods()

[1] "connectome"      "logfc"           "natmi"           "sca"            
 [5] "cellphonedb"     "cytotalk"        "call_squidpy"    "call_cellchat"  
 [9] "call_connectome" "call_sca"        "call_italk"      "call_natmi"

In [5]:
if(!dir.exists("scdb")) dir.create("scdb/")
scdb_init("scdb/", force_reinit=T)
#> initializing scdb to testdb/

initializing scdb to scdb/



In [6]:
mat = scdb_mat("all_cells_liana")
mc_neutrophil = scdb_mc("neutrophil_mc")
mc_kinetics = scdb_mc("kinetics_mc")
pic_mc = scdb_mc("merged_neutrophil_epithelial_mc")

In [7]:
doublet_order = c(2, 20)
doublet_clusters = unique(mc_kinetics@annots)[doublet_order]
lymphoid_order = c(32, 33, 34, 35, 26, 28, 27, 25, 24, 36, 31)
myeloid_order = c(39, 40, 37, 42, 44, 41, 43, 29, 23, 30, 38)
epithelial_order = c(13, 17, 14, 18, 22, 19, 21, 12, 16, 15)
stromal_order = c(1, 3, 10, 11, 9)
fibroblast_order = c(6, 4, 5, 7, 8)
order_index = c(lymphoid_order, myeloid_order, epithelial_order, stromal_order, fibroblast_order)
cell_order = unique(mc_kinetics@annots)[order_index]

In [8]:
cell_order = cell_order[cell_order != "Neutrophils"]

In [9]:
cell_order

[1] "T cells"              "CD8 T cells"          "Treg"                
 [4] "gd T cells"           "gd NKT Ccl5 l"        "gd NKT Ccl5 h"       
 [7] "NKT cells"            "NK cells"             "Cytotoxic NK"        
[10] "ILC2"                 "B cells"              "Monocytes"           
[13] "MonMacs"              "Perivasc macs"        "Ductal macs"         
[16] "cDC1"                 "cDC2"                 "pDC"                 
[19] "Mreg DCs"             "Mast cells"           "Basophils"           
[22] "Progenitor 1"         "Progenitor 2"         "Alveolar"            
[25] "HS"                   "Myoepithelial"        "Basal"               
[28] "MyoLum"               "Alveolar spp l"       "Alveolar spp1 h"     
[31] "Alveolar secretory"   "Endothelial"          "Mcam Endothelial"    
[34] "Smooth muscle"        "Pericytes"            "Schwann cells"       
[37] "fibroblasts"          "Dpp4 fibroblasts"     "IFN Dpp4 fibroblasts"
[40] "CAFs"                 "Pre-adipocytes"

In [10]:
sce = scm_export_mat_to_sce("all_cells_liana", add_log_counts=T)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 25.7 GiB”


In [11]:
good_pics = readLines('annotations/neutrophil_good_pics.txt')

In [12]:
sum(good_pics %in% colnames(sce))

[1] 643

In [13]:
length(good_pics)

[1] 643

In [14]:
sce = sce[,colnames(sce) %in% union(union(names(mc_neutrophil@mc), 
                                          names(mc_kinetics@mc[mc_kinetics@annots[mc_kinetics@mc] %in% cell_order])),
                                    good_pics)]

In [15]:
sum(good_pics %in% colnames(sce))

[1] 643

In [16]:
carcinoma_sce = sce[,sce$condition == 'tumor' & sce$Age %in% c('10w', '12w')]

In [17]:
sum(grepl("__", carcinoma_sce$annots))

[1] 0

In [18]:
mle_res = read.delim("annotations/neutrophil_mle_res.txt", stringsAsFactors = F, row.names=1)

In [19]:
cell_order

[1] "T cells"              "CD8 T cells"          "Treg"                
 [4] "gd T cells"           "gd NKT Ccl5 l"        "gd NKT Ccl5 h"       
 [7] "NKT cells"            "NK cells"             "Cytotoxic NK"        
[10] "ILC2"                 "B cells"              "Monocytes"           
[13] "MonMacs"              "Perivasc macs"        "Ductal macs"         
[16] "cDC1"                 "cDC2"                 "pDC"                 
[19] "Mreg DCs"             "Mast cells"           "Basophils"           
[22] "Progenitor 1"         "Progenitor 2"         "Alveolar"            
[25] "HS"                   "Myoepithelial"        "Basal"               
[28] "MyoLum"               "Alveolar spp l"       "Alveolar spp1 h"     
[31] "Alveolar secretory"   "Endothelial"          "Mcam Endothelial"    
[34] "Smooth muscle"        "Pericytes"            "Schwann cells"       
[37] "fibroblasts"          "Dpp4 fibroblasts"     "IFN Dpp4 fibroblasts"
[40] "CAFs"                 "Pre-adipocytes"

In [20]:
cancer_cells = c("Alveolar spp l", "Alveolar spp1 h", "Alveolar secretory")

In [21]:
tans = c("TAN2", "TAN3")

In [22]:
#good_pics = rownames(mle_res)
alpha = mle_res[good_pics, "alpha"]; names(alpha) = good_pics
epithelial_mc = mle_res[good_pics, "a_mc"]; names(epithelial_mc) = good_pics
immune_mc = mle_res[good_pics, "b_mc"]; names(immune_mc) = good_pics
merged_names = paste(ifelse(pic_mc@annots[immune_mc] %in% tans, "TANs", pic_mc@annots[immune_mc]), 
                     ifelse(pic_mc@annots[epithelial_mc] %in% cancer_cells, 'AlvTumor', pic_mc@annots[epithelial_mc]), 
                     sep = '__')
names(merged_names) = good_pics
#simple_names = paste("Neutrophils", ifelse(pic_mc@annots[epithelial_mc] %in% cancer_cells,
#                                           "AlvTumor",
#                                           pic_mc@annots[epithelial_mc]), sep = '__'); names(simple_names) = good_pics

In [23]:
#c('AB172_D8', 'AB172_D2', 'AB172_D10', 'AB172_E10', 'AB318_O6', 'AB319_B24', 'AB392_A7') %in% good_pics

In [24]:
# Partial merging
#cell_names = factor(ifelse(colnames(carcinoma_sce) %in% mc_neutrophil@cell_names,
#                           "Neutrophils",
#                           ifelse(mc_kinetics@annots[mc_kinetics@mc[colnames(carcinoma_sce)]] %in% cancer_cells,
#                                  "AlvTumor",
#                                  ifelse(colnames(carcinoma_sce) %in% good_pics,
#                                         simple_names[colnames(carcinoma_sce)],
#                                         mc_kinetics@annots[mc_kinetics@mc[colnames(carcinoma_sce)]]))))

In [25]:
cancer_singlets = names(mc_kinetics@mc[mc_kinetics@annots[mc_kinetics@mc] %in% cancer_cells])

In [26]:
tan_singlets = names(mc_neutrophil@mc[mc_neutrophil@annots[mc_neutrophil@mc] %in% tans])

In [27]:
cell_names = factor(ifelse(colnames(carcinoma_sce) %in% good_pics,
                           merged_names[colnames(carcinoma_sce)],
                           ifelse(colnames(carcinoma_sce) %in% names(mc_neutrophil@mc),
                                  ifelse(colnames(carcinoma_sce) %in% tan_singlets,
                                                  'TANs',
                                                  mc_neutrophil@annots[mc_neutrophil@mc[colnames(carcinoma_sce)]]),
                                  ifelse(colnames(carcinoma_sce) %in% cancer_singlets,
                                                  'AlvTumor',
                                                  mc_kinetics@annots[mc_kinetics@mc[colnames(carcinoma_sce)]]))))

In [28]:
cell_names %>% unique %>% sort

[1] Alveolar                          AlvTumor                         
 [3] AP high                           AP high__Alveolar                
 [5] AP high__AlvTumor                 AP high__HS                      
 [7] AP high__Myoepithelial            AP high__MyoLum                  
 [9] AP high__Progenitor 1             AP high__Progenitor 2            
[11] B cells                           Basal                            
[13] Basophils                         CAFs                             
[15] CD8 T cells                       cDC1                             
[17] cDC2                              Cystatin high                    
[19] Cystatin high__AlvTumor           Cystatin high__Progenitor 2      
[21] Cytotoxic NK                      Dpp4 fibroblasts                 
[23] Ductal macs                       Endothelial                      
[25] fibroblasts                       gd NKT Ccl5 h                    
[27] gd NKT Ccl5 l                     gd T cells                       
[29] HS                                IFN Dpp4 fibroblasts             
[31] ILC2                              Mast cells                       
[33] Mcam Endothelial                  MonMacs                          
[35] Monocytes                         Mreg DCs                         
[37] Myoepithelial                     MyoLum                           
[39] NK cells                          NKT cells                        
[41] pDC                               Pericytes                        
[43] Perivasc macs                     Progenitor 1                     
[45] Progenitor 2                      Ptgs2 high                       
[47] Ptgs2 high__Alveolar              Ptgs2 high__AlvTumor             
[49] Ptgs2 high__Myoepithelial         Ptgs2 high__MyoLum               
[51] Ptgs2 high__Progenitor 1          Ptgs2 high__Progenitor 2         
[53] Schwann cells                     Smooth muscle                    
[55] T cells                           TAN1                             
[57] TAN1__Alveolar                    TAN1__AlvTumor                   
[59] TAN1__HS                          TAN1__Myoepithelial              
[61] TAN1__MyoLum                      TAN1__Progenitor 1               
[63] TAN1__Progenitor 2                TANs                             
[65] TANs__Alveolar                    TANs__AlvTumor                   
[67] TANs__HS                          TANs__Myoepithelial              
[69] TANs__MyoLum                      TANs__Progenitor 1               
[71] TANs__Progenitor 2                Treg                             
[73] Young Cystatin high               Young Cystatin high__AlvTumor    
[75] Young Cystatin high__Progenitor 1
75 Levels: Alveolar AlvTumor AP high AP high__Alveolar ... Young Cystatin high__Progenitor 1

In [29]:
write_csv(data.frame(cell=colnames(carcinoma_sce), name=cell_names), "annotations/liana_cell_tanmerged_names.csv")

In [30]:
carcinoma_sce@colData$annots = cell_names

In [31]:
sum(good_pics %in% colnames(carcinoma_sce))

[1] 576

In [32]:
sum(grepl("__", carcinoma_sce$annots))

[1] 576

In [33]:
table(carcinoma_sce@colData$annots) %>% sort


                      AP high__HS            AP high__Myoepithelial 
                                1                                 1 
                        Basophils       Cystatin high__Progenitor 2 
                                1                                 1 
                    Schwann cells                    TAN1__Alveolar 
                                1                                 1 
              TAN1__Myoepithelial                      TAN1__MyoLum 
                                1                                 1 
              TANs__Myoepithelial                             Basal 
                                1                                 2 
        Ptgs2 high__Myoepithelial                Ptgs2 high__MyoLum 
                                2                                 2 
                         TAN1__HS                TAN1__Progenitor 1 
                                2                                 2 
Young Cystatin high__Progenitor 1

In [42]:
carcinoma_sce = carcinoma_sce[,table(carcinoma_sce@colData$annots)[carcinoma_sce@colData$annots] >= 30]

Warning message in NSBS(i, x, exact = exact, strict.upper.bound = !allow.append, :
“subscript is an array, passing it thru as.vector() first”


In [43]:
logcounts(carcinoma_sce) = as(logcounts(carcinoma_sce, withDimnames = FALSE), "dgCMatrix")

In [5]:
liana_path <- system.file(package = "liana")

In [6]:
op_resource <- select_resource("Consensus")[[1]]

In [7]:
ortholog_resource <- generate_homologs(op_resource = op_resource,
                                       .missing_fun = str_to_title,
                                       max_homologs = 5,
                                       target_organism = 10090) # mouse

Warning message:
“Automatic coercion from integer to character was deprecated in purrr 1.0.0.
ℹ Please use an explicit call to `as.character()` within `map_chr()` instead.
ℹ The deprecated feature was likely used in the OmnipathR package.
  Please report the issue at <https://github.com/saezlab/OmnipathR/issues>.”
One-to-many homolog matches: C4A; C4B; CCL27; CD200R1L; CD55; CEACAM1; CLEC10A; CSF2RB; DSG1; ENO1; GSTP1; HLA-DMB; HLA-E; HLA-F; IFNL3; IL11RA; IL22; KIR3DL3; KLRB1; KNG1; LILRB4; LYZ; MRGPRX2; PILRB; PRND; PRSS3; RNASE2; SAA1; SERPINA1



In [8]:
ortholog_resource[ortholog_resource$target_genesymbol == 'Sirpa',]

source_genesymbol,target_genesymbol,source,target,category_intercell_source,database_intercell_source,category_intercell_target,database_intercell_target,sources,references
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Csf1,Sirpa,P09603,P78324,cell_surface_ligand,CellPhoneDB,receptor,connectomeDB2020;CellPhoneDB;ICELLNET;CellTalkDB,CellPhoneDB;CellTalkDB;I2D_CellPhoneDB,CellTalkDB:32103204
Cd47,Sirpa,Q08722,P78324,cell_surface_ligand,connectomeDB2020,receptor,connectomeDB2020;CellPhoneDB;ICELLNET;CellTalkDB,Adhesome;Baccin2019;CellPhoneDB;CellPhoneDB_Cellinker;Cellinker;HPRD;ICELLNET;InnateDB;Wang;Wojtowicz2020;connectomeDB2020,Adhesome:12972543;Adhesome:16081415;Adhesome:17070842;Adhesome:18045614;Adhesome:18633106;Baccin2019:29777034;CellPhoneDB:23602662;Cellinker:15294972;Cellinker:15842360;Cellinker:22511785;HPRD:16291597;HPRD:9872987;ICELLNET:15294972;ICELLNET:17070842;InnateDB:16081415;InnateDB:17070842;connectomeDB2020:10572074
Vwf,Sirpa,P04275,P78324,ecm,CellChatDB,receptor,connectomeDB2020;CellPhoneDB;ICELLNET;CellTalkDB,CellTalkDB;Fantom5_LRdb;LRdb;STRING_talklr;iTALK;talklr,CellTalkDB:32196115
Vegfa,Sirpa,P15692,P78324,ligand,connectomeDB2020;CellPhoneDB;ICELLNET;CellChatDB;CellTalkDB,receptor,connectomeDB2020;CellPhoneDB;ICELLNET;CellTalkDB,CellTalkDB;Fantom5_LRdb;LRdb;STRING_talklr;iTALK;talklr,CellTalkDB:32196115
Sftpd,Sirpa,P35247,P78324,ligand,connectomeDB2020;CellTalkDB,receptor,connectomeDB2020;CellPhoneDB;ICELLNET;CellTalkDB,CellTalkDB;Cellinker;LRdb;connectomeDB2020,CellTalkDB:22511785;Cellinker:22511785;LRdb:22511785;connectomeDB2020:22511785


In [47]:
ortholog_resource[ortholog_resource$target_genesymbol == 'Mcam',]

source_genesymbol,target_genesymbol,source,target,category_intercell_source,database_intercell_source,category_intercell_target,database_intercell_target,sources,references
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Ntn1,Mcam,O95631,P43121,ligand,connectomeDB2020;CellTalkDB,receptor,connectomeDB2020;CellChatDB;CellTalkDB,connectomeDB2020,connectomeDB2020:25656845
Wnt5a,Mcam,P41221,P43121,ligand,connectomeDB2020;CellPhoneDB;ICELLNET;CellChatDB;CellTalkDB,receptor,connectomeDB2020;CellChatDB;CellTalkDB,CellChatDB;CellTalkDB;Cellinker;EMBRACE;Fantom5_LRdb;LRdb;Ramilowski2015;connectomeDB2020;iTALK;talklr,CellChatDB:24335906;CellTalkDB:24335906;Cellinker:24335906;LRdb:24335906;Ramilowski2015:24335906;connectomeDB2020:24335906;talklr:24335906
Lgals3,Mcam,P17931,P43121,,,,,Cellinker,Cellinker:28364041


In [48]:
ortholog_resource[ortholog_resource$source_genesymbol == 'S100a8',]

source_genesymbol,target_genesymbol,source,target,category_intercell_source,database_intercell_source,category_intercell_target,database_intercell_target,sources,references
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
S100a8,Tlr4,P05109,O00206,ligand,connectomeDB2020;CellTalkDB,receptor,connectomeDB2020;CellChatDB;CellTalkDB,CellTalkDB;EMBRACE;Fantom5_LRdb;LRdb;Ramilowski2015;Reactome_LRdb;SIGNOR;connectomeDB2020;iTALK;talklr,CellTalkDB:17767165;LRdb:17767165;Ramilowski2015:17767165;SIGNOR:28137827;connectomeDB2020:17767165;talklr:17767165
S100a8,Itgb2,P05109,P05107,ligand,connectomeDB2020;CellTalkDB,receptor,connectomeDB2020;CellPhoneDB;ICELLNET;CellTalkDB,CellTalkDB,CellTalkDB:9570563
S100a8,Cd36,P05109,P16671,ligand,connectomeDB2020;CellTalkDB,receptor,connectomeDB2020;CellPhoneDB;CellChatDB;CellTalkDB,CellTalkDB,CellTalkDB:31794767
S100a8,Cd68,P05109,P34810,ligand,connectomeDB2020;CellTalkDB,receptor,CellTalkDB,CellTalkDB,CellTalkDB:27312849
S100a8,Ager,P05109,Q15109,ligand,connectomeDB2020;CellTalkDB,receptor,CellPhoneDB;CellTalkDB,CellTalkDB;Cellinker;SIGNOR,CellTalkDB:28504650;Cellinker:28504650;SIGNOR:28137827
S100a8,Cd69,P05109,Q07108,,,,,CellTalkDB,CellTalkDB:26296369


In [49]:
ortholog_resource = ortholog_resource %>% 
    rbind(c('S100a8', 'Mcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2')) %>%
    rbind(c('S100a9', 'Mcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2')) %>%
    rbind(c('S100a8', 'Alcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2')) %>%
    rbind(c('S100a9', 'Alcam', '', '', 'cell_surface_ligand', 'manual', 'receptor', 'manual', '', 
            'https://doi.org/10.1007/s10585-016-9801-2'))


In [50]:
#write_csv(liana_trunc[liana_trunc$source == "Neutrophils" | liana_trunc$target == "Neutrophils",], "neutrophils_lr.csv")

annot_df <- read.csv("../kinetics/annotations/mc_annots.csv", sep=",")
annot_df$label = trimws(annot_df$label)

doublet_order = c(12, 41)
doublet_clusters = unique(annot_df$label)[doublet_order]
lymphoid_order = c(13, 14, 11, 4, 5, 3, 2, 6, 10, 9)
myeloid_order = c(18, 17, 15, 16, 20, 23, 22, 21, 7, 1, 8, 19)
epithelial_order = c(40, 43, 42, 38, 39, 34, 36, 37, 35)
stromal_order = c(24, 33, 32, 31)
fibroblast_order = c(26, 27, 25, 28, 29, 30)

order_index = c(lymphoid_order, myeloid_order, epithelial_order, stromal_order, fibroblast_order)

cell_order = c(unique(annot_df$label)[order_index], 'AlvTumor')

test_sce = carcinoma_sce[,(carcinoma_sce@colData$annots %in% cell_order | grepl("Neutrophils__", carcinoma_sce@colData$annots))]

cell_order

unique(carcinoma_sce$annots) %>% sort

unique(test_sce$annots) %>% sort

In [53]:
carcinoma_sce$annots = droplevels(carcinoma_sce$annots)

In [61]:
liana_res <- liana_wrap(carcinoma_sce, idents_col = "annots", expr_prop=0.05,
                        resource = 'custom', # resource has to be set to 'custom' to work with external resources
                        external_resource = ortholog_resource , # provide orthologous resource
#                        method=c('sca', 'natmi') # run only with sca and natmi for comp. time
                        )


Running LIANA with `annots` as labels!

Warning message in exec(output, ...):
“10599 genes and/or 0 cells were removed as they had no counts!”
LIANA: LR summary stats calculated!

Now Running: Natmi

Now Running: Connectome

Now Running: Logfc

Now Running: Sca

Now Running: Cellphonedb



In [63]:
liana_res = liana_aggregate(liana_res)

Now aggregating natmi

Now aggregating connectome

Now aggregating logfc

Now aggregating sca

Now aggregating cellphonedb

Aggregating Ranks



In [64]:
write_csv(liana_res, "annotations/liana_results_tanmerged_tumor_pics.csv")

In [65]:
saveRDS(carcinoma_sce, "h5ads/liana_carcinoma_tanmerged.RDS")

In [40]:
carcinoma_sce = sce[,sce$condition == 'tumor' & sce$Age %in% c('8w', '10w', '12w')]

In [41]:
merged_names = paste(pic_mc@annots[immune_mc], 
                     ifelse(pic_mc@annots[epithelial_mc] %in% cancer_cells, 'AlvTumor', pic_mc@annots[epithelial_mc]), 
                     sep = '__')
names(merged_names) = good_pics

In [42]:
# Partial merging
cell_names = factor(ifelse(colnames(carcinoma_sce) %in% mc_neutrophil@cell_names,
                           mc_neutrophil@annots[mc_neutrophil@mc[colnames(carcinoma_sce)]],
                           ifelse(mc_kinetics@annots[mc_kinetics@mc[colnames(carcinoma_sce)]] %in% cancer_cells,
                                  "AlvTumor",
                                  ifelse(colnames(carcinoma_sce) %in% good_pics,
                                         merged_names[colnames(carcinoma_sce)],
                                         mc_kinetics@annots[mc_kinetics@mc[colnames(carcinoma_sce)]]))))

In [43]:
write_csv(data.frame(cell=colnames(carcinoma_sce), name=cell_names), "annotations/liana_cell_halfmerged_names_full.csv")

In [44]:
carcinoma_sce@colData$annots = cell_names

In [45]:
table(carcinoma_sce@colData$annots) %>% sort


      Cystatin high__Progenitor 2                    Pre-adipocytes 
                                1                                 1 
              TAN1__Myoepithelial                    TAN2__Alveolar 
                                1                                 1 
                         TAN2__HS                      TAN2__MyoLum 
                                1                                 1 
               TAN2__Progenitor 2               TAN3__Myoepithelial 
                                1                                 1 
                            Basal                         Basophils 
                                2                                 2 
                   TAN1__Alveolar                          TAN1__HS 
                                2                                 2 
                     TAN1__MyoLum                TAN1__Progenitor 1 
                                2                                 2 
                   TAN3__Alveolar

In [46]:
carcinoma_sce = carcinoma_sce[,table(carcinoma_sce@colData$annots)[carcinoma_sce@colData$annots] >= 20]

Warning message in NSBS(i, x, exact = exact, strict.upper.bound = !allow.append, :
“subscript is an array, passing it thru as.vector() first”


In [47]:
logcounts(carcinoma_sce) = as(logcounts(carcinoma_sce, withDimnames = FALSE), "dgCMatrix")

In [ ]:
liana_res <- liana_wrap(carcinoma_sce, idents_col = "annots",
                        resource = 'custom', # resource has to be set to 'custom' to work with external resources
                        external_resource = ortholog_resource , # provide orthologous resource
#                        method=c('sca', 'natmi') # run only with sca and natmi for comp. time
                        )

liana_res = liana_aggregate(liana_res)

write_csv(liana_res, "annotations/liana_results_halfmerged_tumor_pics.csv")

Running LIANA with `annots` as labels!

Warning message in exec(output, ...):
“9357 genes and/or 0 cells were removed as they had no counts!”
Warning message:
“`invoke()` is deprecated as of rlang 0.4.0.
Please use `exec()` or `inject()` instead.
This warning is displayed once every 8 hours.”
Warning message in FUN(...):
“no within-block comparison between AP high__Alveolar and Alveolar”
Warning message in FUN(...):
“no within-block comparison between AP high__Alveolar and AlvTumor”
Warning message in FUN(...):
“no within-block comparison between AP high__Alveolar and AP high”
Warning message in FUN(...):
“no within-block comparison between AP high__AlvTumor and AP high__Alveolar”
Warning message in FUN(...):
“no within-block comparison between AP high__HS and Alveolar”
Warning message in FUN(...):
“no within-block comparison between AP high__HS and AlvTumor”
Warning message in FUN(...):
“no within-block comparison between AP high__HS and AP high”
Warning message in FUN(...):
“no withi

Warning message in FUN(...):
“no within-block comparison between CD8 T cells and AP high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between CD8 T cells and Basal”
Warning message in FUN(...):
“no within-block comparison between CD8 T cells and Basophils”
Warning message in FUN(...):
“no within-block comparison between cDC1 and AP high__Alveolar”
Warning message in FUN(...):
“no within-block comparison between cDC1 and AP high__HS”
Warning message in FUN(...):
“no within-block comparison between cDC1 and AP high__Myoepithelial”
Warning message in FUN(...):
“no within-block comparison between cDC1 and AP high__MyoLum”
Warning message in FUN(...):
“no within-block comparison between cDC1 and AP high__Progenitor 1”
Warning message in FUN(...):
“no within-block comparison between cDC1 and AP high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between cDC1 and Basal”
Warning message in FUN(...):
“no within-block comparison between cDC1 an

Warning message in FUN(...):
“no within-block comparison between Cytotoxic NK and Cystatin high”
Warning message in FUN(...):
“no within-block comparison between Cytotoxic NK and Cystatin high__AlvTumor”
Warning message in FUN(...):
“no within-block comparison between Cytotoxic NK and Cystatin high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between Dpp4 fibroblasts and Alveolar”
Warning message in FUN(...):
“no within-block comparison between Dpp4 fibroblasts and AlvTumor”
Warning message in FUN(...):
“no within-block comparison between Dpp4 fibroblasts and AP high”
Warning message in FUN(...):
“no within-block comparison between Dpp4 fibroblasts and AP high__Alveolar”
Warning message in FUN(...):
“no within-block comparison between Dpp4 fibroblasts and AP high__AlvTumor”
Warning message in FUN(...):
“no within-block comparison between Dpp4 fibroblasts and AP high__HS”
Warning message in FUN(...):
“no within-block comparison between Dpp4 fibroblasts and AP 

Warning message in FUN(...):
“no within-block comparison between gd NKT Ccl5 l and Cystatin high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between gd NKT Ccl5 l and Dpp4 fibroblasts”
Warning message in FUN(...):
“no within-block comparison between gd T cells and AP high__Alveolar”
Warning message in FUN(...):
“no within-block comparison between gd T cells and AP high__HS”
Warning message in FUN(...):
“no within-block comparison between gd T cells and AP high__Myoepithelial”
Warning message in FUN(...):
“no within-block comparison between gd T cells and AP high__MyoLum”
Warning message in FUN(...):
“no within-block comparison between gd T cells and AP high__Progenitor 1”
Warning message in FUN(...):
“no within-block comparison between gd T cells and AP high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between gd T cells and Basal”
Warning message in FUN(...):
“no within-block comparison between gd T cells and Basophils”
Warning me

Warning message in FUN(...):
“no within-block comparison between MonMacs and Cystatin high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between MonMacs and Dpp4 fibroblasts”
Warning message in FUN(...):
“no within-block comparison between Monocytes and AP high__Alveolar”
Warning message in FUN(...):
“no within-block comparison between Monocytes and AP high__HS”
Warning message in FUN(...):
“no within-block comparison between Monocytes and AP high__Myoepithelial”
Warning message in FUN(...):
“no within-block comparison between Monocytes and AP high__MyoLum”
Warning message in FUN(...):
“no within-block comparison between Monocytes and AP high__Progenitor 1”
Warning message in FUN(...):
“no within-block comparison between Monocytes and AP high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between Monocytes and Basal”
Warning message in FUN(...):
“no within-block comparison between Monocytes and Basophils”
Warning message in FUN(...):
“

...

Warning message in FUN(...):
“no within-block comparison between TAN3__Progenitor 2 and Ptgs2 high__MyoLum”
Warning message in FUN(...):
“no within-block comparison between TAN3__Progenitor 2 and Ptgs2 high__Progenitor 1”
Warning message in FUN(...):
“no within-block comparison between TAN3__Progenitor 2 and Ptgs2 high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between TAN3__Progenitor 2 and Schwann cells”
Warning message in FUN(...):
“no within-block comparison between TAN3__Progenitor 2 and Smooth muscle”
Warning message in FUN(...):
“no within-block comparison between TAN3__Progenitor 2 and T cells”
Warning message in FUN(...):
“no within-block comparison between TAN3__Progenitor 2 and TAN1”
Warning message in FUN(...):
“no within-block comparison between TAN3__Progenitor 2 and TAN1__Alveolar”
Warning message in FUN(...):
“no within-block comparison between TAN3__Progenitor 2 and TAN1__AlvTumor”
Warning message in FUN(...):
“no within-block comparison be

Warning message in FUN(...):
“no within-block comparison between Young Cystatin high and Cystatin high__AlvTumor”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high and Cystatin high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high and Cytotoxic NK”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high and Dpp4 fibroblasts”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high and Ductal macs”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high and Endothelial”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high and fibroblasts”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high and gd NKT Ccl5 h”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high and gd NKT Ccl5 l”
Warning message in FUN(...):
“no within-bloc

Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__AlvTumor and cDC2”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__AlvTumor and Cystatin high”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__AlvTumor and Cystatin high__AlvTumor”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__AlvTumor and Cystatin high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__AlvTumor and Cytotoxic NK”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__AlvTumor and Dpp4 fibroblasts”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__AlvTumor and Ductal macs”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__AlvTumor and Endothelial”
Warning message in FUN(...):
“no within-block comparison between Young Cyst

Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and AP high__MyoLum”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and AP high__Progenitor 1”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and AP high__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and B cells”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and Basal”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and Basophils”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and CAFs”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and CD8 T cells”
Warning message in FUN(...):
“no within-block comparison between Yo

Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and TAN3__Progenitor 1”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and TAN3__Progenitor 2”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and Treg”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and Young Cystatin high”
Warning message in FUN(...):
“no within-block comparison between Young Cystatin high__Progenitor 1 and Young Cystatin high__AlvTumor”
LIANA: LR summary stats calculated!

Now Running: Natmi

Now Running: Connectome



In [ ]:
saveRDS(carcinoma_sce, "h5ads/liana_carcinoma_halfmerged.RDS")